In [24]:
from sage.all import *
import sys
import hashlib
import random
import math
from itertools import chain

In [13]:
key_id = '0'
data_path = './data/'
key_suffix = '80' + '_' + key_id
# gmsk = load(data_path + 'sk' + key_suffix)
pk = load(data_path + 'pk')
sig = load(data_path + 'sig' + key_suffix)

In [ ]:
show(pk)

In [30]:
lib_path = './lib/'
load(lib_path + 'common.sage')
load(lib_path + 'goppacode.sage')
load(lib_path + 'Join_Impl.sage')
# load(lib_path + 'KeyPrepare.sage')

In [ ]:
# gpk,gsk = Join(U,B,A,S_A,P_A,H_Goppa,g1,X,SyndromeCalculator,L,2048,170,0.01,1)

In [26]:
key_id = '0'
data_path = './data/'
key_suffix = '80' + '_' + key_id


gpk = load(data_path + 'gpk'+ key_suffix)
gsk = load(data_path + 'gsk'+ key_suffix)

In [8]:
import math

def my_log2(p):
    return math.log(p)/math.log(2)

def log_comb(n, k):
    if not hasattr(log_comb, 'dict'):
        log_comb.dict = dict()
    my_dict = log_comb.dict
    vlu = my_dict.get((n,k), -1)
    if vlu < 0:
        my_dict[(n,k)]=vlu=sum([my_log2(n-j)-my_log2(j+1) for j in range(k)])
    return vlu

def compute_truncate_pr(n, p, lb, ub):
    '''
    Compute sum_{lb<=j<=ub}binom{n}{j}p^j (1-p)^{n-j}
    '''
    return sum([2**(log_comb(n, j)+j*my_log2(p)+(n-j)*my_log2(1-p)) for j in range(lb, ub+1)])

def compute_rejection_sampling_pr(n, p, s, t):
    '''
    Compute the ratio of binomial distribution over shifted binomial distribution.
    '''
    log_comb_n_s = log_comb(n, s)
    return 1/sum([2**(log_comb(t, j)+log_comb(n-t, s-j)-log_comb_n_s+(s-2*j)*my_log2(p)+(2*j-s)*my_log2(1-p)) for j in range(s+1)])


def rejection_sampling_rate(N, s, tau, Xi):
    rate_dict = dict()

    lb = int(N*tau)-Xi; ub = int(N*tau)+Xi

    # denominator of truncated binomial distribution density function
    dem_trun_pr = compute_truncate_pr(N, tau, lb, ub)
    for t in range(lb, ub+1):
        # compute the rejection rate for t
        rejection_sampling_pr = compute_rejection_sampling_pr(N,tau,s,t)/dem_trun_pr
        rate_dict[t] = rejection_sampling_pr
    M = max(rate_dict.values())
    return rate_dict, M

In [9]:
def hash_function_1(n, y,msg,field):
    r = int(hashlib.sha512((str(y)+msg).encode('utf-8')).hexdigest(), base=16)
#     s = set()
    lst = n*[0]
    lst = vector(field,lst)
    while r != 0:
        ele = r%n
        r = r//n
        lst[ele] = lst[ele] + 1
    return lst


In [10]:
def hash_function_2(n, w, t, y, msg, field):
    r = int(hashlib.sha512((str(t)+str(y)+msg).encode('utf-8')).hexdigest(), base=16)
    s = set()
    while len(s) < w and r != 0:
        s.add(r%n)
        r = r//n
    if len(s) >= w:
        lst = n*[0]
        for ele in s:
            lst[ele] = 1
        lst = vector(field,lst)
        return lst
    else:
        print ('Error in hash function')


In [46]:
def sign( msg,n,k,l, w,sigma_1,sigma_2, tau,xi,pk,gsk,field):
#     random.seed(msg)
    X_i1 = gsk[0]
    X_i2 = gsk[1]
#     U = pk[0]
    B = pk[1]
    A = pk[2]
    G_pub = pk[3]
    
#     R=PolynomialRing(GF(2), 'x', implementation='NTL');x=R.gen();S=R.quotient(x^n-1, 'a');a = S.gen()
#     h = S(pk)
#     s1_lst, s2_lst = sk
#     s1 = S(s1_lst); s2 = S(s2_lst)
    counter = 0
    eta_1 = 0.5 * (1-(1-2*sigma_1)**w)
    eta_2 = 0.5 * (1-(1-2*sigma_2)**w)
    zeta_1 = tau + eta_1 - 2 * tau * eta_1
    zeta_2 = tau + eta_2 - 2 * tau * eta_2
    a_1 = n * zeta_1 - xi
    b_1 = n * zeta_1 + xi
    a_2 = n * zeta_2 - xi
    b_2 = n * zeta_2 + xi
    print(a_1,b_1,a_2,b_2)
    while True:
        counter = counter + 1
#         if counter > 5000:
#             print ('Run signing too many times')
        
        e1 = gen_binom_vec(n, tau,field)
        e2 = gen_binom_vec(n, tau,field)
        y1 = e1 * B.transpose()
        y2 = e1 * B.transpose() + e2 * A.transpose()
        c1 = hash_function_1(l, y1, msg,field)
        r  = random_word(k - l, field)
        err = random_error(n, 32, field)
        t = vector(chain(r,c1)) * G_pub + err
        c2 = hash_function_2(l,w,t, y2, msg,field)
        z1 = e1 + c2 * X_i1.transpose()
        z2 = e2 + c2 * X_i2.transpose()
        wt_z1 = get_weight(z1)
        wt_z2 = get_weight(z2)
#         print(wt_z1,wt_z2)
 
        
        # compute rejection rate
        rate_dict1, M1 = rejection_sampling_rate(n, get_weight(c2 * X_i1.transpose()), zeta_1, xi)
        rate_dict2, M2 = rejection_sampling_rate(n, get_weight(c2 * X_i2.transpose()), zeta_2, xi)
        

        if wt_z1 < int(a_1) or wt_z1 > int(b_1) or wt_z2 < int(a_2) or wt_z2 > int(b_2):
            continue

        if random.random() < rate_dict1[wt_z1]/M1 and random.random() < rate_dict2[wt_z2]/M2:
            print(counter)
            return (t, y2, z1, z2, msg)


In [37]:
n = 2048
k = 1696
l = 170
sigma_1 = 0.009
sigma_2 = 0.013
w = 17
xi = 20
r = 1696
m = 2048
tau = 0.09


# U = pk[0]
# B = pk[1]
# A = pk[2]
# G_pub = pk[3]
eta_1 = 0.5 * (1-(1-2*sigma_1)**w)
eta_2 = 0.5 * (1-(1-2*sigma_2)**w)
zeta_1 = tau + eta_1 - 2 * tau * eta_1
zeta_2 = tau + eta_2 - 2 * tau * eta_2
a_1 = n * zeta_1 - xi
b_1 = n * zeta_1 + xi
a_2 = n * zeta_2 - xi
b_2 = n * zeta_2 + xi

In [39]:
def gen_binom_vec(nn, tau, field):
    v= [1 if random.random()<tau else 0 for _ in range(nn)]
    v = vector(field,v)
    return v

In [ ]:
msg = 'INK'
sig = sign(msg,n,k,l,w,sigma_1,sigma_2,tau,xi,pk,gsk,GF(2))

In [ ]:
show(sig[0])

In [ ]:
# print(getsize(A.str()))
# print(getsize(A.list()))

In [ ]:
print(get_weight(sig[2]))

In [ ]:
# print(len(vecmat))

In [ ]:
key_id = '0'
data_path = './data/'
key_suffix = '80' + '_' + key_id


filename = data_path + 'sig' + key_suffix
print('secret key saved in file ' + filename + '.sobj')
save(sig, filename)


In [6]:
def verify(sig,pk,n,k,l,w,sigma_1,sigma_2,tau,xi,field):
    t = sig[0]
    y2 = sig[1]
    z1 = sig[2]
    z2 = sig[3]
    msg = sig[4]
    U = pk[0]
    B = pk[1]
    A = pk[2]
    eta_1 = 0.5 * (1-(1-2*sigma_1)**w)
    eta_2 = 0.5 * (1-(1-2*sigma_2)**w)
    zeta_1 = tau + eta_1 - 2 * tau * eta_1
    zeta_2 = tau + eta_2 - 2 * tau * eta_2
    a_1 = n * zeta_1 - xi
    b_1 = n * zeta_1 + xi
    a_2 = n * zeta_2 - xi
    b_2 = n * zeta_2 + xi
    
#     y2 = vector(field,y2)
    c2 = hash_function_2(l,w,t, y2, msg,field)
    y2_prime = z1 * B.transpose() + z2 * A.transpose() - c2 * U.transpose()
#     y2_prime = vector(field,y2_prime)
    c2_prime = hash_function_2(l,w,t, y2_prime, msg,field)
    
    wt_z1 = get_weight(z1)
    wt_z2 = get_weight(z2)
#     print(y2_prime == y2)
#     print(c2_prime != c2)
    if wt_z1 < int(a_1) or wt_z1 > int(b_1) or wt_z2 < int(a_2) or wt_z2 > int(b_2) or c2_prime != c2:
        return 0
    
    return 1
    

In [14]:
print(verify(sig,pk,n,k,l,w,sigma_1,sigma_2,tau,xi,GF(2)))

0


In [17]:
N = 64

In [18]:
user_i = randint(0,N-1)

In [19]:
user_i

15

In [32]:
key_id = '0'
data_path = './data/'
key_suffix = '80' + '_' + key_id


gpklst = load(data_path + 'gpklst'+ key_suffix)
gsklst = load(data_path + 'gsklst'+ key_suffix)

In [33]:
gpk_i = gpklst[user_i]
gsk_i = gsklst[user_i]

In [44]:
import random
import time

In [47]:
msg = 'INK'
start = time.time()
sig_i = sign(msg,n,k,l,w,sigma_1,sigma_2,tau,xi,pk,gsk_i,GF(2))
end = time.time()
print('签名生成程序运行时间为：', end - start, '秒')

387.3920761746913 427.3920761746913 467.4427395015503 507.4427395015503
97
签名生成程序运行时间为： 32.157705545425415 秒


In [48]:
start = time.time()
print(verify(sig_i,pk,n,k,l,w,sigma_1,sigma_2,tau,xi,GF(2)))
end = time.time()
print('签名验证程序运行时间为：', end - start, '秒')

1
签名验证程序运行时间为： 0.05981779098510742 秒


In [49]:
key_id = '0'
data_path = './data/'
key_suffix = '80' + '_' + key_id
gmsk = load(data_path + 'sk' + key_suffix)
# pk = load(data_path + 'pk')

In [51]:
S = gmsk[5]
P = gmsk[6]
b = gmsk[7]
g = gmsk[8]

G_pub = pk[3]
G_Goppa = S.inverse() * G_pub * P.transpose()

In [52]:
r = 1696
m = 2048
n = 2048
t = 32
#     show(U)
#     show(B)
#     show(A)
#     show(g1)

F = b.parent()
PR = g.parent()
Y = PR.gen()
K_ = GF(2)
L = [F(b**i) for i in range(2048)]
SyndromeCalculator = matrix(PR, 1, len(L));
for i in range(len(L)):
    SyndromeCalculator[0, i] = (Y - L[i]).inverse_mod(g);




In [53]:
def openSig(sig,pk,gpklst,g,X,SyndromeCalculator,L,G_Goppa,S,P,l,w,N,field):  
    t = sig[0]
    y2 = sig[1]
    z1 = sig[2]
    z2 = sig[3]
    msg = sig[4]
    B = pk[1]
    
    word = t * P.transpose()
    word_mat = matrix(K_,word)
#         yd = decodePatterson(word_mat,g1,X,SyndromeCalculator,L)
    word_prime = decodePatterson(word_mat,g,X,SyndromeCalculator,L)
    word_prime = vector(K_,word_prime.list())
    m = G_Goppa.solve_left(word_prime)
    m = m * S.inverse()
    
    c1 = m[1696-170:]
    c1 = vector(field,c1)
    c2 = hash_function_2(l,w,t, y2, msg,field)
    
    for i in range(N):
        G_i = gpklst[i]
        if c1 == hash_function_1(l,z1*B.transpose()-c2*G_i.transpose(), msg,field):
            return i
    
    return -1
    
    
    

In [57]:
N = 2 ** 6
start = time.time()
openSig(sig_i,pk,gpklst,g,Y,SyndromeCalculator,L,G_Goppa,S,P,l,w,N,GF(2))
end = time.time()
print('签名成员身份打开程序运行时间为：', end - start, '秒')

签名成员身份打开程序运行时间为： 1.1083929538726807 秒
